In [ ]:
# ────────────────────────────────────────────────────────────────────────────────────
# MILESTONE 2 – MODULE-3 : Tranning THE FILE WITH YOLOv8 AND RESULT ANALYSIS
# ────────────────────────────────────────────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ────────────────────────────────────────────────
#  PCB Defect Detection CODE With YOLO
# ────────────────────────────────────────────────


!pip install ultralytics -q

import os
import shutil
import xml.etree.ElementTree as ET
import yaml
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# ────────────────────────────────────────────────
# CONFIG
# ────────────────────────────────────────────────
BASE_DIR = "/content/drive/MyDrive/PCB_DATASET/"
IMAGES_ROOT = os.path.join(BASE_DIR, "images")
IMAGES_COMBINED = os.path.join(BASE_DIR, "images_combined")
ANNOTATIONS_DIR = os.path.join(BASE_DIR, "Annotations")
YOLO_DATASET_DIR = os.path.join(BASE_DIR, "yolo_dataset")
YOLO_YAML_PATH = os.path.join(BASE_DIR, "pcb_yolo_dataset.yaml")
MODEL_PATH = os.path.join(BASE_DIR, "best_yolov8_pcb.pt")
OUTPUT_DIR = os.path.join(BASE_DIR, "results/yolo_red_boxes/")

class_names = [
    'missing_hole',
    'mouse_bite',
    'open_circuit',
    'short',
    'spur',
    'spurious_copper'
]

num_classes = len(class_names)
class_to_idx = {name: i for i, name in enumerate(class_names)}

os.makedirs(IMAGES_COMBINED, exist_ok=True)
os.makedirs(YOLO_DATASET_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# ────────────────────────────────────────────────
# 1. Copy images to images_combined
# ────────────────────────────────────────────────
print("Copying images from defect folders → images_combined/")
copied = 0
for defect in ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']:
    src_folder = os.path.join(IMAGES_ROOT, defect)
    if os.path.exists(src_folder):
        for fname in os.listdir(src_folder):
            if fname.lower().endswith('.jpg'):
                src = os.path.join(src_folder, fname)
                dst = os.path.join(IMAGES_COMBINED, fname)
                if not os.path.exists(dst):
                    shutil.copy(src, dst)
                    copied += 1
print(f"Copied {copied} images. Total in combined: {len(os.listdir(IMAGES_COMBINED))}")

Copying images from defect folders → images_combined/
Copied 693 images. Total in combined: 693


In [ ]:
# ────────────────────────────────────────────────
# 2. Collect images from images_combined
# ────────────────────────────────────────────────
all_images = [os.path.join(IMAGES_COMBINED, f) for f in os.listdir(IMAGES_COMBINED) if f.lower().endswith('.jpg')]
unique_images = list(set(all_images))
print(f"Unique images: {len(unique_images)}")
if len(unique_images) == 0:
    raise ValueError("No images found in images_combined/")

train_files, test_val = train_test_split(unique_images, test_size=0.3, random_state=42)
val_files, test_files = train_test_split(test_val, test_size=0.5, random_state=42)

Unique images: 693


In [ ]:
# ────────────────────────────────────────────────
# 3. Prepare YOLO folders
# ────────────────────────────────────────────────
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(YOLO_DATASET_DIR, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(YOLO_DATASET_DIR, split, 'labels'), exist_ok=True)

In [ ]:
# ────────────────────────────────────────────────
# 4. Convert XML → YOLO labels
# ────────────────────────────────────────────────
def convert_xml_to_yolo(xml_path, img_w, img_h):
    if not os.path.exists(xml_path):
        return []
    tree = ET.parse(xml_path)
    root = tree.getroot()
    labels = []
    for obj in root.findall('object'):
        name_elem = obj.find('name')
        if name_elem is None or not name_elem.text:
            continue
        raw_name = name_elem.text.strip()
        norm_name = raw_name.lower().replace(' ', '_').replace('-', '_')
        cls_id = class_to_idx.get(norm_name)
        if cls_id is None:
            continue
        bndbox = obj.find('bndbox')
        if bndbox is None:
            continue
        try:
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)
        except:
            continue
        cx = (xmin + xmax) / 2 / img_w
        cy = (ymin + ymax) / 2 / img_h
        w = (xmax - xmin) / img_w
        h = (ymax - ymin) / img_h
        labels.append(f"{cls_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")
    return labels

processed = 0
for split, files in [('train', train_files), ('val', val_files), ('test', test_files)]:
    print(f"\n{split.upper()} ({len(files)} images):")
    for img_path in files:
        filename = os.path.basename(img_path)
        xml_path = None
        for defect in ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']:
            cand = os.path.join(ANNOTATIONS_DIR, defect, filename.replace('.jpg', '.xml'))
            if os.path.exists(cand):
                xml_path = cand
                break
        if not xml_path:
            continue
        try:
            img = Image.open(img_path)
            w, h = img.size
            labels = convert_xml_to_yolo(xml_path, w, h)
            if not labels:
                continue
            shutil.copy(img_path, os.path.join(YOLO_DATASET_DIR, split, 'images', filename))
            lbl_path = os.path.join(YOLO_DATASET_DIR, split, 'labels', filename.replace('.jpg', '.txt'))
            with open(lbl_path, 'w') as f:
                f.write('\n'.join(labels))
            processed += 1
            print(f"Processed: {filename}")
        except:
            pass
print(f"\nTotal processed images with labels: {processed}")


TRAIN (485 images):
Processed: 08_missing_hole_06.jpg
Processed: 04_missing_hole_15.jpg
Processed: 01_mouse_bite_04.jpg
Processed: 04_missing_hole_07.jpg
Processed: 08_short_07.jpg
Processed: 04_spurious_copper_10.jpg
Processed: 08_mouse_bite_04.jpg
Processed: 01_mouse_bite_05.jpg
Processed: 04_spurious_copper_14.jpg
Processed: 11_short_05.jpg
Processed: 11_spurious_copper_02.jpg
Processed: 05_spur_03.jpg
Processed: 07_mouse_bite_05.jpg
Processed: 08_short_08.jpg
Processed: 10_open_circuit_03.jpg
Processed: 08_spur_10.jpg
Processed: 04_missing_hole_03.jpg
Processed: 10_missing_hole_01.jpg
Processed: 07_open_circuit_10.jpg
Processed: 11_spurious_copper_10.jpg
Processed: 01_short_08.jpg
Processed: 06_spurious_copper_04.jpg
Processed: 04_missing_hole_04.jpg
Processed: 05_spurious_copper_06.jpg
Processed: 06_spurious_copper_07.jpg
Processed: 08_spur_05.jpg
Processed: 01_spurious_copper_01.jpg
Processed: 07_spurious_copper_10.jpg
Processed: 04_open_circuit_14.jpg
Processed: 01_short_06.jpg

In [ ]:
# ────────────────────────────────────────────────
# 5. dataset.yaml
# ────────────────────────────────────────────────
dataset_yaml = {
    'path': YOLO_DATASET_DIR,
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': num_classes,
    'names': class_names
}
with open(YOLO_YAML_PATH, 'w') as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False)

In [ ]:
# ────────────────────────────────────────────────
#  Training with YOLO
# ────────────────────────────────────────────────
model = YOLO('yolov8n.pt')
results = model.train(
    data=YOLO_YAML_PATH,
    epochs=80,
    imgsz=640,
    batch=16,
    patience=15,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=os.path.join(BASE_DIR, "runs"),
    name='pcb_yolo',
    exist_ok=True
)

best_weights = os.path.join(BASE_DIR, "runs/pcb_yolo/weights/best.pt")
if os.path.exists(best_weights):
    shutil.copy(best_weights, MODEL_PATH)

Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/PCB_DATASET/pcb_yolo_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=pcb_yolo, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15,

In [ ]:
# ────────────────────────────────────────────────
# Validation & Result Analysis
# ────────────────────────────────────────────────
print("\n" + "="*65)
print(" VALIDATION & RESULT ANALYSIS ".center(65, "="))
print("="*65)

model = YOLO(MODEL_PATH)
val_results = model.val(data=YOLO_YAML_PATH, split='val', plots=True, save=True)

# Show per-class metrics
print("\nPer-class performance:")
print("Class              Precision  Recall   F1-Score  Support")
print("-"*60)
p = val_results.box.p
r = val_results.box.r
f1 = val_results.box.f1

for i, name in enumerate(class_names):
    print(f"{name:18} {p[i]:9.3f} {r[i]:9.3f} {f1[i]:9.3f}    {int(val_results.confusion_matrix.matrix[:, i].sum() if hasattr(val_results, 'confusion_matrix') else '?')}")

print(f"\nmAP@0.5    : {val_results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {val_results.box.map:.4f}")

print("\nCheck folder for plots:")
print(f"  {os.path.join(BASE_DIR, 'runs/pcb_defects')}")
print("  → confusion_matrix.png, results.png, PR_curve.png, etc.")


================== VALIDATION & RESULT ANALYSIS =================
Ultralytics 8.4.5 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 73 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 1.0±0.5 ms, read: 70.0±40.7 MB/s, size: 1367.5 KB)
val: Scanning /content/drive/MyDrive/PCB_DATASET/yolo_dataset/val/labels.cache... 104 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 104/104 1.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 4.6s/it 32.0s
                   all        104        452      0.872      0.726      0.809      0.393
          missing_hole         16         68      0.991      0.971      0.992      0.594
            mouse_bite         18         81      0.751      0.605      0.715      0.342
          open_circuit         15         67      0.796      0.746      0.807      0.356
                 short         19         76  